# Predicting Sale Prices Using Advanced Regression and Feature Engineering

In this notebook I have explored the House Prices dataset from Kaggle using several regression algorithms.
We’ll go through the complete machine learning workflow step by step:
- 🧹 Data exploration & cleaning

- ⚙️ Feature engineering

- 🧠 Model building & tuning

- 🤝 Model blending (XGBoost, LightGBM, Ridge, Lasso)

- 📊 Generating final predictions

### The end goal is to achieve a competitive relative error in  prediction of the Saleprice of Houses with maximum accuracy.

We begin by importing essential libraries for data handling, visualization, and modeling.
This includes `Pandas, NumPy, Seaborn, Matplotlib`, and multiple machine learning libraries like `Scikit-learn`, `XGBoost`, `LightGBM`, and `CatBoost`.

In [ ]:
# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

# Data handling
import numpy as np
import pandas as pd

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Modeling
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
import xgboost as xgb
import lightgbm as lgb
import catboost as cb

# Display settings
pd.set_option('display.float_format', lambda x: '%.3f' % x)

# Load data
train = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/train.csv')
test = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/test.csv')

print("Train shape:", train.shape)
print("Test shape:", test.shape)
train.head()

Key **observations** after loading the datasets:

- Train set has 1460 rows and 81 columns

- Test set has 1459 rows and 80 columns

- SalePrice is our target variable, available only in the training set.

# Initial Data Exploration

We examine the dataset to understand its structure and identify missing values.

- Many categorical columns like `PoolQC`, `Fence`, `Alley`, `MiscFeature` have large sections of missing data.

- Numeric features like `LotFrontage`, `GarageYrBlt`, and `MasVnrArea` have moderate missing values.

In [ ]:
# Overview of training data
print("----- TRAIN DATA INFO -----")
train.info()

print("\n----- TEST DATA INFO -----")
test.info()



In [ ]:
# Count missing values per column
missing = train.isnull().sum()
missing = missing[missing > 0].sort_values(ascending=False)

print(f"Total columns with missing values: {len(missing)}")
missing.head(20)


#### We will summarize numerical statistics using `.describe()` to check data spread, means, and possible outliers.

In [ ]:
train.describe().T.head(15)

## Understanding the Target Variable `(Saleprice)`:
**Insights:**
- The price distribution is right-skewed — most houses are moderately priced with a few expensive outliers.

- 75% of all houses cost below ~214,000 USD.

- Mean > Median confirms a right-skewed distribution.

In [ ]:
plt.figure(figsize=(8,5))
sns.histplot(train['SalePrice'],kde=True)
plt.title('Distribution of saleprice of Houses')
plt.show()
print(train['SalePrice'].describe())


In [ ]:
corr = train.select_dtypes(include=[np.number]).corr()['SalePrice'].sort_values(ascending=False)
corr.head(15)

The above step also helps us realize why a **log transformation** will later improve model stability and reduce skewness.

# Visual Exploratory Data Analysis (EDA) for (Numeric features)
We compute correlations between numeric features and SalePrice.
Top correlated features:

- `OverallQual`

- `GrLivArea`

- `GarageCars`

- `TotalBsmtSF`

- `1stFlrSF`

  
Let's visualize how the top correlated numeric features relate to **SalePrice**.


In [ ]:
top_features = ['OverallQual', 'GrLivArea', 'GarageCars', 'GarageArea','TotalBsmtSF', '1stFlrSF', 'FullBath', 'TotRmsAbvGrd', 'YearBuilt']

plt.figure(figsize=(16, 10))
for i, feature in enumerate(top_features[:6]):
    plt.subplot(2, 3, i + 1)
    sns.scatterplot(data=train, x=feature, y='SalePrice', alpha=0.7)
    plt.title(f'{feature} vs SalePrice')
plt.tight_layout()
plt.show()


Above, we can already see that:
- Higher **OverallQual** and **GrLivArea** strongly increase SalePrice.
- There are potential **outliers** — very large houses sold at lower prices.


# Outlier Handling & Categorical EDA

## Removing outliers:

In [ ]:
# Identify potential outliers
outliers = train[(train['GrLivArea'] > 4000) & (train['SalePrice'] < 300000)]
display(outliers[['Id', 'GrLivArea', 'SalePrice']])

# Remove them from the training set
train = train.drop(outliers.index)

print(f"New train shape after removing outliers: {train.shape}")

We detected **extreme outliers**: houses with GrLivArea > 4000 but low SalePrice < 300000.

These can harm the model and introduce bias, so we remove them to improve model generalization.

## Exploring categorical data:

We will visualize relationships between categorical variables and SalePrice using **Boxplots**:

In [ ]:
# Plot settings for exploring Categorical features
plt.figure(figsize=(14,6))
sns.boxplot(x='Neighborhood', y='SalePrice', data=train)
plt.xticks(rotation=45)
plt.title('SalePrice distribution across Neighborhoods')
plt.show()


**Insights:**

After reviewing the above boxplot it can be seen that:

- `Neighborhood`: Some neighborhoods consistently have higher prices.


In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14,6))

sns.boxplot(x='HouseStyle', y='SalePrice', data=train, ax=axes[0])
axes[0].set_title('HouseStyle vs SalePrice')
axes[0].tick_params(axis='x', rotation=45)

sns.boxplot(x='SaleCondition', y='SalePrice', data=train, ax=axes[1])
axes[1].set_title('SaleCondition vs SalePrice')
axes[1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()


**Insights 🧠**  
- `HouseStyle`: 2Story and 1Story homes tend to have higher prices, while 1.5Fin and 1.5Unf styles are cheaper.  
- `SaleCondition`: 'Partial' (new construction) sales are the most expensive.

  
These patterns confirm **categorical features are strong predictors**.


# Feaure Engineering and Data Prep

### Handling Missing Values and Imputation:

- Columns where NaN means “None” (e.g., `Alley`, `Fence`, `PoolQC`) are filled with "`None`".

- Numeric basement/garage values are replaced with `0`.

- LotFrontage is filled using the **median** per Neighborhood.

- GarageYrBlt missing → `0`.

- Electrical missing → value from **mode**.

In [ ]:
import numpy as np
import pandas as pd

train = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/train.csv')
test = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/test.csv')


# Workin on a copy
df_train = train.copy()

# 1) Columns where NaN means "None"
none_cols = [
    'Alley','BsmtQual','BsmtCond','BsmtExposure','BsmtFinType1','BsmtFinType2',
    'FireplaceQu','GarageType','GarageFinish','GarageQual','GarageCond',
    'PoolQC','Fence','MiscFeature','MasVnrType'
]
for c in none_cols:
    if c in df_train.columns:
        df_train[c] = df_train[c].fillna('None')

# Basement/garage 
zero_cols = ['BsmtFinSF1','BsmtFinSF2','BsmtUnfSF','TotalBsmtSF',
             'BsmtFullBath','BsmtHalfBath','GarageCars','GarageArea','MasVnrArea']
for c in zero_cols:
    if c in df_train.columns:
        df_train[c] = df_train[c].fillna(0)

# LotFrontage: impute by Neighborhood median
if 'LotFrontage' in df_train.columns and 'Neighborhood' in df_train.columns:
    df_train['LotFrontage'] = df_train.groupby('Neighborhood')['LotFrontage'].transform(
        lambda s: s.fillna(s.median())
    )

# GarageYrBlt: missing means no garage
if 'GarageYrBlt' in df_train.columns:
    df_train['GarageYrBlt'] = df_train['GarageYrBlt'].fillna(0)

# Electrical: single missing
if 'Electrical' in df_train.columns:
    df_train['Electrical'] = df_train['Electrical'].fillna(df_train['Electrical'].mode()[0])

# sanity check of remaining NA
na_left = df_train.isna().sum()
na_left = na_left[na_left>0].sort_values(ascending=False)
print("Still missing after rules:")
print(na_left)


Above output shows that there are **no missing values** anymore.

## Log-Transforming the target variable `SalePrice`:

We apply log transformation to SalePrice using np.log1p() bcause,

- it reduces the influence of extremely high-priced houses.
- Makes data more normally distributed.
- Helps linear models handle target variability better.

In [ ]:
# Create a new target variable (log-transformed SalePrice)
y = np.log1p(train['SalePrice'])   # log(1 + SalePrice)

# Drop SalePrice and Id to get features
X = train.drop(columns=['SalePrice', 'Id'])

# Check transformation visually
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(8,5))
sns.histplot(y, kde=True, color='skyblue')
plt.title('Distribution of Log-Transformed SalePrice')
plt.show()

print("y mean:", y.mean(), "y std:", y.std())


Now the distribution of the target `SalePrice` above is not right-skewed anymore when it is log-tansformed.

## Encoding the categorical values:

We will separate numeric and categorical columns:

- Apply ordinal mappings to quality-related features (e.g., ExterQual, KitchenQual, BsmtQual, etc.).

- Remaining categorical features are one-hot encoded.

This encoding allows both linear and non linear models to process mixed data types correctly.

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

# Create a copy for safety
X = train.drop(columns=['SalePrice', 'Id']).copy()

# Ordinal mappings (keep meaningful order)
qual_map = {'None':0,'Po':1,'Fa':2,'TA':3,'Gd':4,'Ex':5}
bsmt_exp_map = {'None':0,'No':0,'Mn':1,'Av':2,'Gd':3}
bsmt_fin_map = {'None':0,'Unf':1,'LwQ':2,'Rec':3,'BLQ':4,'ALQ':5,'GLQ':6}
paved_map = {'N':0,'P':1,'Y':2}
bin_map = {'N':0,'Y':1}
functional_map = {'Sal':0,'Sev':1,'Maj2':2,'Maj1':3,'Mod':4,'Min2':5,'Min1':6,'Typ':7}

ordinal_maps = {
    'ExterQual': qual_map, 'ExterCond': qual_map,
    'BsmtQual': qual_map, 'BsmtCond': qual_map, 'BsmtExposure': bsmt_exp_map,
    'BsmtFinType1': bsmt_fin_map, 'BsmtFinType2': bsmt_fin_map,
    'HeatingQC': qual_map, 'KitchenQual': qual_map,
    'FireplaceQu': qual_map, 'GarageQual': qual_map, 'GarageCond': qual_map,
    'PoolQC': qual_map, 'PavedDrive': paved_map, 'CentralAir': bin_map,
    'Functional': functional_map
}

for col, mapping in ordinal_maps.items():
    if col in X.columns:
        X[col] = X[col].map(mapping).astype('float64')

# Split remaining columns by dtype
num_cols = X.select_dtypes(include=['int64', 'float64']).columns.tolist()
cat_cols = X.select_dtypes(include=['object']).columns.tolist()

print(f"Numeric columns: {len(num_cols)} | Categorical columns: {len(cat_cols)}")

# Preprocess: imputers + one-hot encoder
numeric_proc = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median'))
])

categorical_proc = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

preprocess = ColumnTransformer(
    transformers=[
        ('num', numeric_proc, num_cols),
        ('cat', categorical_proc, cat_cols)
    ]
)

print("Encoding setup complete — ready for model training.")

# Linear Model Training
## Baseline model training using Ridge Regression:

In [ ]:
# Baseline Ridge Regression with 10-Fold Cross-Validation

from sklearn.linear_model import Ridge
from sklearn.model_selection import KFold, cross_val_score
from sklearn.pipeline import Pipeline
import numpy as np

# Create a Ridge regression model inside a pipeline
ridge = Pipeline(steps=[
    ('preprocessor', preprocess),
    ('model', Ridge(alpha=10.0, random_state=42))
])

# Define 10-fold cross-validation setup
cv = KFold(n_splits=10, shuffle=True, random_state=42)

# Evaluate using Root Mean Squared Log Error
scores = cross_val_score(
    ridge, X, y,
    scoring='neg_root_mean_squared_error',
    cv=cv,
    n_jobs=-1
)

print(f"Ridge 10-Fold CV log-RMSE: {-scores.mean():.4f} ± {scores.std():.4f}")


**Insights:**

- CV log-RMSE: 0.1374 ± 0.0379

- This means we have now achieved a **`13.7%` relative prediction error**

## Hyperparameter tuning
Now that we have the baseline model ready, we can test multiple alpha (regularization) values using `RidgeCV` and `LassoCV`

In [ ]:
from sklearn.linear_model import RidgeCV, LassoCV

# Define range of alpha values to test
alphas = np.logspace(-3, 3, 50)  # from 0.001 to 1000

# RidgeCV
ridge_cv = RidgeCV(alphas=alphas, scoring='neg_root_mean_squared_error', cv=10)
ridge_cv.fit(preprocess.fit_transform(X), y)
print(f"Best Ridge alpha: {ridge_cv.alpha_:.4f}")

#LassoCV for comparison
lasso_cv = LassoCV(alphas=alphas, cv=10, random_state=42, max_iter=10000)
lasso_cv.fit(preprocess.fit_transform(X), y)
print(f"Best Lasso alpha: {lasso_cv.alpha_:.4f}")


**Insights:**
- Ridge α ≈ `10.9854`
- Lasso α ≈ `0.0010`

Also note that the std deviation is still `0.0379`, we will also try to improve this.

## Rechecking Cross-Validation:

In [ ]:
from sklearn.model_selection import KFold, cross_val_score
from sklearn.linear_model import Ridge, Lasso
from sklearn.pipeline import Pipeline
import numpy as np

best_alpha_ridge = 10.9854
best_alpha_lasso = 0.0010

ridge_best = Pipeline([
    ('preprocessor', preprocess),
    ('model', Ridge(alpha=best_alpha_ridge, random_state=42))
])

lasso_best = Pipeline([
    ('preprocessor', preprocess),
    ('model', Lasso(alpha=best_alpha_lasso, random_state=42, max_iter=20000))
])

cv = KFold(n_splits=10, shuffle=True, random_state=42)

ridge_scores = cross_val_score(ridge_best, X, y,
                               scoring='neg_root_mean_squared_error',
                               cv=cv, n_jobs=-1)
lasso_scores = cross_val_score(lasso_best, X, y,
                               scoring='neg_root_mean_squared_error',
                               cv=cv, n_jobs=-1)

print(f"Ridge (α={best_alpha_ridge:.4f})  CV: {-ridge_scores.mean():.4f} ± {ridge_scores.std():.4f}")
print(f"Lasso (α={best_alpha_lasso:.4f})  CV: {-lasso_scores.mean():.4f} ± {lasso_scores.std():.4f}")

**Insights:**

- Ridge CV log-RMSE: `0.1374`

- Lasso CV log-RMSE: `0.1375`

Hence, it can be noted that both perform similary when checked with cross validation.

## Fine-tuning lasso again 
- to check if the true best alpha(α) for Lasso is even smaller `(<0.001)`

For this, we will test with smaller alpha values raging from `0.00001 to 0.01`

In [ ]:
from sklearn.linear_model import LassoCV
import numpy as np

# Transform once for speed
X_mm = preprocess.fit_transform(X)

alphas_fine = np.logspace(-5, -2, 30)  # 1e-5 ... 1e-2
lasso_cv_fine = LassoCV(alphas=alphas_fine, cv=10, random_state=42, max_iter=30000)
lasso_cv_fine.fit(X_mm, y)

best_alpha_lasso_refined = float(lasso_cv_fine.alpha_)
print("Refined best Lasso alpha:", best_alpha_lasso_refined)


Because the refined Lasso printed an alpha <0.001, we will re-evaluate Lasso with the new alpha `0.0005736152510448681`

In [ ]:
from sklearn.model_selection import KFold, cross_val_score
from sklearn.linear_model import Lasso
from sklearn.pipeline import Pipeline

lasso_best = Pipeline([
    ('preprocessor', preprocess),
    ('model', Lasso(alpha=best_alpha_lasso_refined, random_state=42, max_iter=30000))
])

cv = KFold(n_splits=10, shuffle=True, random_state=42)
lasso_scores = cross_val_score(lasso_best, X, y,
                               scoring='neg_root_mean_squared_error',
                               cv=cv, n_jobs=-1)
print(f"Lasso (α={best_alpha_lasso_refined:.6f}) CV: {-lasso_scores.mean():.4f} ± {lasso_scores.std():.4f}")


- So with the updated alpha `α=0.000574` that is far lesser than the older one, it confirms that a smaller **α** helped the Lasso model to learn more detail without overfitting
- We also improved the relative error in prediction to `0.1342` **(approx. 13.4%)** which was earlier `0.1375`

**Hence, we can say that this is our best performing linear model.**

# Training and Evaluating Non Linear models 
Now, We will  train the non linear models **`LightGBM`** and **`XGBoost`** with early stopping and CV and then generate OOF (out-of-fold) predictions for both models.

In [ ]:
import numpy as np
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error

# Reuse preprocessed features
X_mm = preprocess.fit_transform(X)

def rmse_log(y_true, y_pred):
    return np.sqrt(np.mean((y_true - y_pred)**2))

kf = KFold(n_splits=10, shuffle=True, random_state=42)

# ----- LightGBM -----
import lightgbm as lgb
lgb_params = dict(
    n_estimators=5000,
    learning_rate=0.01,
    num_leaves=31,
    subsample=0.8,
    colsample_bytree=0.8,
    reg_alpha=0.0,
    reg_lambda=0.0,
    random_state=42
)

oof_lgb = np.zeros(len(X))
lgb_best_iters = []

for tr, va in kf.split(X_mm):
    X_tr, X_va = X_mm[tr], X_mm[va]
    y_tr, y_va = y.iloc[tr], y.iloc[va]

    model_lgb = lgb.LGBMRegressor(**lgb_params)
    model_lgb.fit(
        X_tr, y_tr,
        eval_set=[(X_va, y_va)],
        eval_metric='rmse',
        callbacks=[lgb.early_stopping(stopping_rounds=200, verbose=False)]
    )
    oof_lgb[va] = model_lgb.predict(X_va, num_iteration=model_lgb.best_iteration_)
    lgb_best_iters.append(model_lgb.best_iteration_)

rmse_lgb = rmse_log(y, oof_lgb)
print(f"LightGBM 10-fold OOF log-RMSE: {rmse_lgb:.4f} (avg best_iter ≈ {int(np.mean(lgb_best_iters))})")

# ----- XGBoost -----
from xgboost import XGBRegressor
xgb_params = dict(
    n_estimators=6000,
    learning_rate=0.01,
    max_depth=6,
    subsample=0.8,
    colsample_bytree=0.8,
    reg_alpha=0.0,
    reg_lambda=1.0,
    objective='reg:squarederror',
    random_state=42,
    n_jobs=-1
)

oof_xgb = np.zeros(len(X))
xgb_best_iters = []

for tr, va in kf.split(X_mm):
    X_tr, X_va = X_mm[tr], X_mm[va]
    y_tr, y_va = y.iloc[tr], y.iloc[va]

    model_xgb = XGBRegressor(**xgb_params)
    model_xgb.fit(
        X_tr, y_tr,
        eval_set=[(X_va, y_va)],
        eval_metric='rmse',
        verbose=False,
        early_stopping_rounds=200
    )
    best_iter = model_xgb.best_iteration if model_xgb.best_iteration is not None else xgb_params['n_estimators']
    oof_xgb[va] = model_xgb.predict(X_va, iteration_range=(0, best_iter))
    xgb_best_iters.append(best_iter)

rmse_xgb = rmse_log(y, oof_xgb)
print(f"XGBoost 10-fold OOF log-RMSE: {rmse_xgb:.4f} (avg best_iter ≈ {int(np.mean(xgb_best_iters))})")


**Insights:**

Now we have the OOf predictions for **LightGBM and XGBoost** which is,
- log RMSE: `0.1247` **approximately 12.47%** for LightGBM
- and log RMSE: `0.1241` **approximately 12.41%** for XGBoost

Next step is to combine all four of the models' OOF predictions and blend them together for getting the best possible **Out-OF-Fold (OOF) log RMSE**.

# Generating OOF Predictions for Ridge and Lasso
We have to generate the OOF predictions for our Linear models because we already have the OOF log RMSE for the non linear models (LightGBM and XGBoost)
- we are doing this to match the prediction metrics of all 4 models, so that later we can ensemble them together and get the final OOF log RMSE.
- This will ensure all four models `(Ridge, Lasso, LightGBM, XGBoost)` are evaluated using consistent folds.

In [ ]:
import numpy as np
from sklearn.model_selection import KFold
from sklearn.linear_model import Ridge, Lasso
from sklearn.pipeline import Pipeline

# 10-Fold Cross Validation setup (same folds for all models for fairness)
kf = KFold(n_splits=10, shuffle=True, random_state=42)

# Create Ridge and Lasso pipelines with tuned alphas
ridge_best = Pipeline([
    ('preprocessor', preprocess),
    ('model', Ridge(alpha=10.9854, random_state=42))
])

lasso_best = Pipeline([
    ('preprocessor', preprocess),
    ('model', Lasso(alpha=0.0005736152510448681, random_state=42, max_iter=30000))
])

# Empty arrays to store out-of-fold predictions (same length as training data)
oof_ridge = np.zeros(len(X))
oof_lasso = np.zeros(len(X))

# Generate OOF predictions for Ridge and Lasso
for tr, va in kf.split(X):
    ridge_best.fit(X.iloc[tr], y.iloc[tr])
    oof_ridge[va] = ridge_best.predict(X.iloc[va])

    lasso_best.fit(X.iloc[tr], y.iloc[tr])
    oof_lasso[va] = lasso_best.predict(X.iloc[va])

# RMSE in log space (≈ RMSLE on original scale)
def rmse_log(y_true, y_pred):
    return np.sqrt(np.mean((y_true - y_pred)**2))

ridge_rmse = rmse_log(y, oof_ridge) 
lasso_rmse = rmse_log(y, oof_lasso) 
print("Ridge OOF log RMSE",ridge_rmse), 
print("Lasso OOF log RMSE",lasso_rmse)

Now that we have the values of **Ridge and Lasso** OOF log RMSE above at approximately `14.25%` Ridge and `14.11%` Lasso, 
- We can now combine all four models and blend them to see if the final log RMSE delivers a lower score than all of the models individually.



# Blending All Models 
 
- we will test the best set of weights for the four models by assigning multiple grids of weights for each model.
- we will only choose the set of weights which proves to deliver the **lowest RMSLE** of all.

Note that we will be **using the OOF prediction values for all 4 models**


In [ ]:
# Candidate weights for Ridge, Lasso, LightGBM, XGBoost
candidates = [
    (0.20, 0.20, 0.30, 0.30),
    (0.15, 0.15, 0.35, 0.35),
    (0.10, 0.10, 0.40, 0.40),
    (0.25, 0.25, 0.25, 0.25),
    (0.33, 0.33, 0.17, 0.17),
    (0.10, 0.20, 0.35, 0.35),
    (0.20, 0.10, 0.35, 0.35),
]

best_w = None
best_score = 999

# Test each weight combination and pick the best one (lowest RMSLE)
for w in candidates:
    w_r, w_l, w_lb, w_x = w
    blend = w_r*oof_ridge + w_l*oof_lasso + w_lb*oof_lgb + w_x*oof_xgb
    score = rmse_log(y, blend)
    print(f"Weights {w} → OOF log-RMSE: {score:.4f}")
    if score < best_score:
        best_score, best_w = score, w

print(f"\n Best OOF blend: weights={best_w}, score={best_score:.4f}")

Now after reviewing the final log RMSE scores with respect to each grid of weights assigned after combining all 4 models,
- It can be said that `0.1206` is the best score until now.
- It should also be noted that `0.1206` **(approx.12%)** is a better score than all of the previously predicted OOF log RMSE of each model individually.

We can now move ahead and train our best grid of weights, `weights=(0.1, 0.1, 0.4, 0.4)` on full data.

# Training and Testing on entire data

### Handling a Preprocessing Error Before Final Training:

While preparing the final models for full training and test prediction,  
I encountered a **ValueError** related to missing value imputation:

> `ValueError: Cannot use median strategy with non-numeric data: could not convert string to float: 'TA'`

This occurred because some **ordinal categorical features** (like `ExterQual`, `BsmtQual`, `KitchenQual`, etc.)  
contained string values (`'TA'`, `'Gd'`, `'Ex'`, etc.) in the **test set**,  
which conflicted with the **numeric median imputer** in the preprocessing pipeline.

## Step A: Ordinal Mapping & Combined Preprocessing

In [ ]:
# MAP ORDINALS & FIT PREPROCESSOR ON TRAIN + TEST

import lightgbm as lgb
from xgboost import XGBRegressor
from sklearn.linear_model import Ridge, Lasso

# Use average best iterations from early stopping
avg_lgb_iter = int(np.mean(lgb_best_iters))
avg_xgb_iter = int(np.mean(xgb_best_iters))

# making copies
X_train_fixed = X.copy()
X_test_fixed  = test.drop(columns=['Id']).copy()

# Define the same ordinal maps used earlier
qual_map       = {'Po':1, 'Fa':2, 'TA':3, 'Gd':4, 'Ex':5}
bsmt_exp_map   = {'None':0, 'No':1, 'Mn':2, 'Av':3, 'Gd':4}
bsmt_fin_map   = {'None':0, 'Unf':1, 'LwQ':2, 'Rec':3, 'BLQ':4, 'ALQ':5, 'GLQ':6}
paved_map      = {'N':0, 'P':1, 'Y':2}
bin_map        = {'N':0, 'Y':1}
functional_map = {'Sal':1,'Sev':2,'Maj2':3,'Maj1':4,'Mod':5,'Min2':6,'Min1':7,'Typ':8}

ordinal_maps = {
    'ExterQual': qual_map, 'ExterCond': qual_map,
    'BsmtQual': qual_map, 'BsmtCond': qual_map, 'BsmtExposure': bsmt_exp_map,
    'BsmtFinType1': bsmt_fin_map, 'BsmtFinType2': bsmt_fin_map,
    'HeatingQC': qual_map, 'KitchenQual': qual_map,
    'FireplaceQu': qual_map, 'GarageQual': qual_map, 'GarageCond': qual_map,
    'PoolQC': qual_map, 'PavedDrive': paved_map, 'CentralAir': bin_map,
    'Functional': functional_map
}

def apply_ordinal_maps_inplace(df):
    for col, mp in ordinal_maps.items():
        if col in df.columns:
            if df[col].dtype == 'O' or df[col].dtype.name == 'category':
                df[col] = df[col].fillna('None').map(mp)
            df[col] = df[col].astype('float', errors='ignore')

apply_ordinal_maps_inplace(X_train_fixed)
apply_ordinal_maps_inplace(X_test_fixed)

# Fit existing preprocessor on TRAIN + TEST combined
combined = pd.concat([X_train_fixed, X_test_fixed], axis=0)
preprocess_full = preprocess.fit(combined)

# Transform train and test with fitted preprocessor
X_full    = preprocess_full.transform(X_train_fixed)
X_test_mm = preprocess_full.transform(X_test_fixed)

print("✅ Preprocessing completed. Shapes:", X_full.shape, X_test_mm.shape)


#### Insights:

**The fix for above error:**
- Mapped ordinal strings (like `TA`, `Gd`, `Ex`) to numbers in both train and test sets.

- Refit the preprocessor on combined train + test data to learn all category levels.

- Transformed both datasets again for clean modeling.

## Step B: Train all models on full data

We train all four models on the full dataset using tuned parameters:

- **Ridge** (α = 10.9854)

- **Lasso** (α = 0.000574)

- **LightGBM** (best_iter ≈ 1879)

- **XGBoost** (best_iter ≈ 2251)

In [ ]:
# ==============================
# 🧩 PART 3B — TRAIN FULL MODELS (LIGHTGBM, XGBOOST, RIDGE, LASSO)
# ==============================

# Train LightGBM on full data
lgb_params = dict(
    n_estimators=avg_lgb_iter,
    learning_rate=0.01,
    num_leaves=31,
    subsample=0.8,
    colsample_bytree=0.8,
    reg_alpha=0.0,
    reg_lambda=0.0,
    random_state=42
)
lgb_full = lgb.LGBMRegressor(**lgb_params)
lgb_full.fit(X_full, y)

# Train XGBoost on full data
xgb_params = dict(
    n_estimators=avg_xgb_iter,
    learning_rate=0.01,
    max_depth=6,
    subsample=0.8,
    colsample_bytree=0.8,
    reg_alpha=0.0,
    reg_lambda=1.0,
    objective='reg:squarederror',
    random_state=42,
    n_jobs=-1
)
xgb_full = XGBRegressor(**xgb_params)
xgb_full.fit(X_full, y, verbose=False)

# Train Ridge and Lasso on full data
ridge_full = Ridge(alpha=10.9854, random_state=42)
ridge_full.fit(X_full, y)

lasso_full = Lasso(alpha=0.0005736152510448681, random_state=42, max_iter=30000)
lasso_full.fit(X_full, y)

print("Models trained on full data (Ridge, Lasso, LightGBM, XGBoost).")


#### All models are now blended and trained together successfully.

## Step C: Predicting Test set, Blending final Results and Creating submission file.

In this final step, We will generate predictions for the test set:

- Get log predictions from each model.

- Blend them using the best weights `(0.1, 0.1, 0.4, 0.4)`.

- Convert log predictions back to actual Dollar prices.

In [ ]:
# Get log predictions from each model
pred_ridge_log = ridge_full.predict(X_test_mm)
pred_lasso_log = lasso_full.predict(X_test_mm)
pred_lgb_log   = lgb_full.predict(X_test_mm)
pred_xgb_log   = xgb_full.predict(X_test_mm)

# Blend predictions using best weights
w_r, w_l, w_lb, w_x = best_w  # Example: (0.1, 0.1, 0.4, 0.4)
pred_log = (w_r*pred_ridge_log +
            w_l*pred_lasso_log +
            w_lb*pred_lgb_log +
            w_x*pred_xgb_log)

# Convert log predictions back to actual dollar prices
pred = np.expm1(pred_log)
pred = np.clip(pred, 0, None)

# Create final submission file
sub = test[['Id']].copy()
sub['SalePrice'] = pred
sub.to_csv('submission.csv', index=False)

print(f"\nSaved submission.csv with weights={best_w}, OOF blend score={best_score:.4f}")


Above we finally recieved a competitive OOF blend score of all 4 models, that is: `0.1206` **(approx. 12% error)** 

# 🧠 Key Learnings:
- Regularization improves linear stability (Ridge, Lasso).

- Tree-based models handle complex non-linear relationships better.

- Blending reduces overfitting and improves generalization.

- Proper preprocessing and ordinal mapping are crucial for error-free model pipelines.


# UPDATE - Trying to achieve an even better relative error than `12%`

# Part A — Feature engineering + neighborhood target encoding + preprocess

## Part A.1

In [ ]:
# FEATURE ENGINEERING (applied to BOTH train & test)

import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer

# Start from your existing frames
X_base   = X.copy()
test_base = test.copy()

def add_features(df):
    df = df.copy()

    # 1) Total square footage (finished areas)
    df['TotalSF'] = df.get('TotalBsmtSF', 0) + df.get('1stFlrSF', 0) + df.get('2ndFlrSF', 0)

    # 2) Total bathrooms (full + half baths upstairs + basement)
    df['TotalBath'] = (
        df.get('FullBath', 0)
        + 0.5 * df.get('HalfBath', 0)
        + df.get('BsmtFullBath', 0)
        + 0.5 * df.get('BsmtHalfBath', 0)
    )

    # 3) Ages relative to sale year (how old things are)
    df['Age']       = (df.get('YrSold', 0) - df.get('YearBuilt', 0)).clip(lower=0)
    df['RemodAge']  = (df.get('YrSold', 0) - df.get('YearRemodAdd', 0)).clip(lower=0)
    df['GarageAge'] = (df.get('YrSold', 0) - df.get('GarageYrBlt', 0)).clip(lower=0)

    # 4) Porch/Deck footprint and a binary flag
    porch_cols = ['WoodDeckSF','OpenPorchSF','EnclosedPorch','3SsnPorch','ScreenPorch']
    df['PorchSF']  = df[porch_cols].sum(axis=1)
    df['HasPorch'] = (df['PorchSF'] > 0).astype(int)

    # 5) Interaction: quality × living area (bigger *and* better quality costs more)
    df['Qual_x_GrLiv'] = df.get('OverallQual', 0) * df.get('GrLivArea', 0)

    # 6) Log transforms for very skewed size features (keep originals too)
    for c in ['GrLivArea', 'TotalSF', 'LotArea']:
        if c in df.columns:
            df[f'log1p_{c}'] = np.log1p(df[c])

    # 7) New-build hint from SaleCondition
    df['IsNew'] = (df.get('SaleCondition', 'Normal') == 'Partial').astype(int)

    return df

X_fe    = add_features(X_base)
test_fe = add_features(test_base)
print("Added engineered features. Example new cols:",
      [c for c in ['TotalSF','TotalBath','Age','PorchSF','HasPorch','Qual_x_GrLiv','log1p_GrLivArea'] if c in X_fe.columns][:7])


**Insights:**

- `TotalSF`: because buyers pay for total finished area, not just one floor.

- `TotalBath`: more baths = higher price; half baths count as 0.5.

- `Age` / `RemodAge` / `GarageAge`: newer houses/remodels/garages usually sell higher.

- `PorchSF`/`HasPorch`: outdoor space adds value.

- `Qual×GrLiv`: large houses with high quality get a price “boost” beyond a simple sum.

- `log1p(size)`: normalizes extreme values so models learn smoother relations.

- `IsNew`: SaleCondition='Partial' often means new construction.

## Part A.2 - Target Encoding for Neighborhood (Leak-Free)
In this step, I created a new feature called `TE_Neighborhood`, which assigns each neighborhood the average log sale price based only on training folds.
I used leak-free K-Fold target encoding, meaning each row’s encoded value is computed from other folds and never from itself.


In [ ]:
# LEAK-FREE TARGET ENCODING FOR Neighborhood

from sklearn.model_selection import KFold

# y is your log-transformed target from earlier steps (np.log1p(SalePrice))
kf = KFold(n_splits=10, shuffle=True, random_state=42)
global_mean = y.mean()  # fallback

# Out-of-fold means for train
te_tr = pd.Series(index=X_fe.index, dtype=float)

for tr_idx, val_idx in kf.split(X_fe):
    tr_neigh  = X_fe.loc[tr_idx, 'Neighborhood'].astype(str)
    val_neigh = X_fe.loc[val_idx, 'Neighborhood'].astype(str)
    means = y.iloc[tr_idx].groupby(tr_neigh).mean()  # mean log(SalePrice) per neighborhood in the TRAIN fold only
    te_tr.iloc[val_idx] = val_neigh.map(means).fillna(global_mean)

X_fe['TE_Neighborhood'] = te_tr

# Apply train means to test
train_means_final = y.groupby(X_fe['Neighborhood'].astype(str)).mean()
test_fe['TE_Neighborhood'] = (
    test_fe['Neighborhood'].astype(str).map(train_means_final).fillna(global_mean)
)

print("Target-encoded Neighborhood added as TE_Neighborhood.")


**Insights:**

- Encoding is done using **10-fold CV**

- Training data = out-of-fold neighborhood means

- Test data = neighborhood means computed from full train

- Global mean is used for unseen neighborhood levels


## Part A.3 — Rebuilding Preprocessing After Feature Engineering

Since feature engineering added new numerical features (like `TotalSF`,`Age`, log features, TE features, etc.), I had to refit the entire preprocessing pipeline on both train + test rows combined.

In [ ]:
# APPLY ORDINAL MAPS + PREPROCESS + TRANSFORM

# Reusing ordinal quality maps
qual_map       = {'Po':1, 'Fa':2, 'TA':3, 'Gd':4, 'Ex':5}
bsmt_exp_map   = {'None':0, 'No':1, 'Mn':2, 'Av':3, 'Gd':4}
bsmt_fin_map   = {'None':0, 'Unf':1, 'LwQ':2, 'Rec':3, 'BLQ':4, 'ALQ':5, 'GLQ':6}
paved_map      = {'N':0, 'P':1, 'Y':2}
bin_map        = {'N':0, 'Y':1}
functional_map = {'Sal':1,'Sev':2,'Maj2':3,'Maj1':4,'Mod':5,'Min2':6,'Min1':7,'Typ':8}

ordinal_maps = {
    'ExterQual': qual_map, 'ExterCond': qual_map,
    'BsmtQual': qual_map, 'BsmtCond': qual_map, 'BsmtExposure': bsmt_exp_map,
    'BsmtFinType1': bsmt_fin_map, 'BsmtFinType2': bsmt_fin_map,
    'HeatingQC': qual_map, 'KitchenQual': qual_map,
    'FireplaceQu': qual_map, 'GarageQual': qual_map, 'GarageCond': qual_map,
    'PoolQC': qual_map, 'PavedDrive': paved_map, 'CentralAir': bin_map,
    'Functional': functional_map
}

def apply_ordinal_maps_inplace(df):
    for col, mp in ordinal_maps.items():
        if col in df.columns:
            if df[col].dtype == 'O' or df[col].dtype.name == 'category':
                df[col] = df[col].fillna('None').map(mp)
            df[col] = df[col].astype('float', errors='ignore')

apply_ordinal_maps_inplace(X_fe)
apply_ordinal_maps_inplace(test_fe)

# Build a fresh ColumnTransformer on the engineered frames
numeric_features     = X_fe.select_dtypes(include=[np.number]).columns.tolist()
categorical_features = X_fe.select_dtypes(exclude=[np.number]).columns.tolist()

numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler(with_mean=False))  # safe for sparse outputs downstream
])
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse=True))
])

preprocess_v2 = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features),
    ],
    remainder='drop'
)

# Fit on train+test combined features so encoders see all categories
combined_v2 = pd.concat([X_fe, test_fe], axis=0)
preprocess_v2.fit(combined_v2)

# Transform to model-ready matrices
Xmm       = preprocess_v2.transform(X_fe)
Xmm_test  = preprocess_v2.transform(test_fe)

print("Shapes after FE + TE + preprocess:", Xmm.shape, Xmm_test.shape)


# PART B — Out-of-Fold Predictions for Base Models

## Part B.1 - Training Base Models to Generate OOF Predictions
Here I trained five different base models using 10-fold out-of-fold (OOF) training:

- Ridge Regression

- Lasso Regression

- ElasticNet

- LightGBM

- XGBoost

I used the same fold splits for all models so their OOF predictions align row-by-row.

In [ ]:
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import Ridge, Lasso, ElasticNet
import lightgbm as lgb
from xgboost import XGBRegressor
import numpy as np
import pandas as pd

def rmsle(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

kf = KFold(n_splits=10, shuffle=True, random_state=42)
n = Xmm.shape[0]

# storage
oof = {}
test_preds = {}

def fit_oof(name, model_fn, Xtr, ytr, Xte, folds=kf):
    oof_pred = np.zeros(n)
    te_pred  = np.zeros(Xte.shape[0])
    best_iters = []

    for tr_idx, val_idx in folds.split(Xtr):
        X_tr, X_val = Xtr[tr_idx], Xtr[val_idx]
        y_tr, y_val = y.iloc[tr_idx], y.iloc[val_idx]

        m = model_fn()

        # LightGBM special fitting (early stopping)
        if isinstance(m, lgb.LGBMRegressor):
            m.fit(X_tr, y_tr,
                  eval_set=[(X_val, y_val)],
                  eval_metric='rmse',
                  callbacks=[lgb.early_stopping(200, verbose=False)])
            best_iters.append(m.best_iteration_)
            oof_pred[val_idx] = m.predict(X_val, num_iteration=m.best_iteration_)
            te_pred += m.predict(Xte, num_iteration=m.best_iteration_) / folds.get_n_splits()

        # XGBoost special fitting
        elif isinstance(m, XGBRegressor):
            m.fit(
                X_tr, y_tr,
                eval_set=[(X_val, y_val)],
                eval_metric='rmse',
                verbose=False,
                early_stopping_rounds=200
            )
    
            # In new XGBoost versions:
            best_iters.append(m.best_iteration)

            # OOF predictions using best_iteration
            oof_pred[val_idx] = m.predict(X_val, iteration_range=(0, m.best_iteration))

            # Test predictions averaged across folds
            te_pred += m.predict(Xte, iteration_range=(0, m.best_iteration)) / folds.get_n_splits()


        else:
            # linear models
            m.fit(X_tr, y_tr)
            oof_pred[val_idx] = m.predict(X_val)
            te_pred += m.predict(Xte) / folds.get_n_splits()

    print(f"{name}: OOF log-RMSE = {rmsle(y, oof_pred):.5f}", 
          (f" | avg best_iter ≈ {int(np.mean(best_iters))}" if best_iters else ""))

    oof[name] = oof_pred
    test_preds[name] = te_pred


# --- Linear models (strong tuning) ---
def ridge_model():
    return Ridge(alpha=8.0, random_state=42)

def lasso_model():
    return Lasso(alpha=5.7e-4, max_iter=60000, random_state=42)

def enet_model():
    return ElasticNet(alpha=1e-4, l1_ratio=0.2, max_iter=60000, random_state=42)

fit_oof("Ridge", ridge_model, Xmm, y, Xmm_test)
fit_oof("Lasso", lasso_model, Xmm, y, Xmm_test)
fit_oof("ElasticNet", enet_model, Xmm, y, Xmm_test)

# --- LightGBM (stronger settings) ---
def lgbm_model():
    return lgb.LGBMRegressor(
        learning_rate=0.005,
        n_estimators=12000,
        num_leaves=31,
        min_data_in_leaf=10,
        subsample=0.8,
        colsample_bytree=0.8,
        reg_alpha=0.1,
        reg_lambda=0.5,
        random_state=42
    )

# --- XGBoost (stronger settings) ---
def xgb_model():
    return XGBRegressor(
        learning_rate=0.008,
        n_estimators=15000,
        max_depth=4,
        min_child_weight=1,
        subsample=0.75,
        colsample_bytree=0.75,
        reg_alpha=0.001,
        reg_lambda=1.0,
        objective='reg:squarederror',
        random_state=42,
        n_jobs=-1
    )

fit_oof("LightGBM", lgbm_model, Xmm, y, Xmm_test)
fit_oof("XGBoost",  xgb_model,  Xmm, y, Xmm_test)


## Part B.2 — Building the Stacking Matrices

In [ ]:
import pandas as pd

stack_train = pd.DataFrame({name: oof[name] for name in oof})
stack_test  = pd.DataFrame({name: test_preds[name] for name in test_preds})

print("Stack train shape:", stack_train.shape)
print("Stack test shape:", stack_test.shape)

stack_train.head()


**Insights:**

Using the OOF predictions, I created two new matrices:

`stack_train` → shape (1460, 5)
Contains OOF predictions from all 5 base models for training rows.

`stack_test` → shape (1459, 5)
Contains averaged predictions from all 5 base models for test rows.

These two matrices will be the input to the stacking meta-model in the next part.

# PART C — Final Stacked Meta-Model

In this final step, I trained a **Lasso meta-model** on top of the stacking matrices.
The model learns how to combine Ridge, Lasso, ElasticNet, LightGBM, and XGBoost.

I used 10-fold CV again to generate meta-level OOF predictions and then used the OOf predictions test predictions across folds.

In [ ]:
from sklearn.linear_model import Lasso
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
import numpy as np

def rmsle(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

# 10-fold CV for meta-model
kf = KFold(n_splits=10, shuffle=True, random_state=42)

oof_meta = np.zeros(stack_train.shape[0])     # OOF predictions for meta-model
test_meta = np.zeros(stack_test.shape[0])     # Test predictions averaged over folds

for tr_idx, val_idx in kf.split(stack_train):
    
    # Training part of stacking features
    X_tr, X_val = stack_train.iloc[tr_idx], stack_train.iloc[val_idx]
    y_tr, y_val = y.iloc[tr_idx], y.iloc[val_idx]

    # Meta-model (Lasso works incredibly well here)
    meta_model = Lasso(alpha=0.0005, max_iter=50000, random_state=42)
    meta_model.fit(X_tr, y_tr)

    # OOF prediction for validation fold
    oof_meta[val_idx] = meta_model.predict(X_val)

    # Prediction for test set (accumulated)
    test_meta += meta_model.predict(stack_test) / kf.get_n_splits()

# Print meta-model performance
print("Meta-model OOF log-RMSE:", rmsle(y, oof_meta))

# Final predictions from stacked model
final_log_preds = test_meta
final_preds = np.expm1(final_log_preds)
final_preds = np.clip(final_preds, 0, None)

# Save submission
sub = test[['Id']].copy()
sub['SalePrice'] = final_preds
sub.to_csv('submission.csv', index=False)

print("\n✅ Final stacked submission saved as submission.csv")


### Model accuracy (Interpreting the final score)

My final stacked meta-model achieved an OOF log-RMSE of **`0.1163`**.  
Hence, this score translates to an approximate **11.6% relative error** in predicting the `SalePrice` of houses.

This means that, on average, my predictions differ from the actual prices by roughly **±11–12%**, which is considered a strong performance for this competition.


**Insights:**

- Log predictions were converted back to actual prices

- Negative values were clipped

- The final CSV **(submission.csv)** was created for Kaggle submission

## 🧠 Key Learnings

- Feature engineering (TotalSF, Age, baths, porches, TE) adds strong predictive signal.
- Leak-free target encoding prevents data leakage and improves model stability.
- Rebuilding preprocessing after FE keeps train/test feature spaces fully aligned.
- OOF predictions create clean inputs for stacking without overfitting.
- Boosting models capture non-linear patterns that linear models miss.
- Stacking blends strengths of all models and reduces variance.
- Lasso meta-model learns the best combination of base model predictions.

